
# Premier TP – Rappels et exercices en Python
**Objectifs** : Ce TP vise à rappeler les bases de Python et à les appliquer à travers des exercices de **statistiques**, de **visualisation graphique** et de **manipulation de données**.

---
##  Python
Python est un langage de programmation polyvalent qui permet de créer et exécuter des algorithmes, qu’ils soient personnalisés ou issus de bibliothèques existantes (via des [bibliothèques logicielles](https://fr.wikipedia.org/wiki/Biblioth%C3%A8que_logicielle)). Gratuit et open source ([logiciel libre](https://fr.wikipedia.org/wiki/Logiciel_libre)), il est accessible à tous.

**Version recommandée** : Nous utiliserons **Python 3** dans ce TP. 

**Outils suggérés pour l’installation** :
Vous pouvez utiliser les outils suivant sur vos ordinateurs personnels :
- [Anaconda](https://www.anaconda.com/download) (distribution complète incluant Jupyter et de nombreuses bibliothèques scientifiques).
- [VSCode](https://code.visualstudio.com)

Les ordinateurs des salles de TP disposent des outils nécessaires pour ces travaux pratiques.

**Remarque** : Python étant un langage développé en anglais, les noms des fonctions et commandes en découlent. Une maîtrise basique de l’anglais technique est donc utile.

## Utilisation de Python pour ce TP

### Environnements de travail
Python dispose d’un interpréteur en ligne de commande (`python`), mais nous privilégierons les notebooks Jupyter pour ce TP. Ceux-ci reproduisent un environnement similaire à *Mathematica*, avec une organisation claire :
- Cellules de code : Pour exécuter des scripts (calculs, graphiques, etc.).
- Cellules Markdown  : Pour rédiger des explications ou des commentaires (syntaxe [Markdown](https://jupyterbook.org/en/stable/reference/cheatsheet.html)).
- Ordre d’exécution flexible : **Attention** ! Cette liberté est une source fréquente d’erreurs et de non-reproductibilité des résultats. **Vérifiez toujours l’ordre logique des cellules et de leur execution.**

**Lancement d’un notebook** :
- Ouvrez un terminal et tapez `jupyter-notebook`.
- Les fichiers ont l’extension `.ipynb` (ex: `mon_TP.ipynb`).

**Alternative** : Pour un éditeur plus complet, installez [VS Code](https://code.visualstudio.com/) (gratuit et compatible avec les notebooks).


## Bonnes pratiques en Python (et avec les notebooks)
Python est un langage puissant mais permissif : une mauvaise utilisation peut nuire à la clarté du code ou introduire des bugs. Voici quelques conseils  :

1. **Importation des modules** :
   - **À éviter** : `from module import *` (risque de **conflits de noms** avec vos variables locales).
   - **À privilégier** :
     ```python
     import module as mod  # Ex: import numpy as np
     mod.fonction()        # Utilisation explicite (ex: np.mean())
     ```
   *→ Cela permet de tracer l’origine des fonctions et d’éviter les ambiguïtés.*

2. **Gestion des variables globales** :
   - **À éviter** : Modifier une variable globale et réévaluer une cellule sans contrôle.
   - **À privilégier** :
     - Encapsuler votre code dans des fonctions (ex: `def calculer_moyenne(data):`).
     - Appeler ces fonctions dans de nouvelles cellules avec les nouvelles valeurs à tester.
   *→ Gain en reproductibilité et en lisibilité.*

3. **Organisation du code** :
   - Découpez votre travail en petites cellules :
     - Une cellule = *une fonction ou une tâche spécifique* (ex: nettoyage des données, calcul d’une statistique).
     - Une autre cellule = *tests ou applications* de cette fonction.
   *→ Évite les réinitialisations coûteuses et les redéfinitions inutiles.*



## Première partie : Intégration Monte Carlo
Pour commencer, nous allons implémenter une méthode classique d’intégration par Monte Carlo. Cet exercice permettra de manipuler Python tout en appliquant des concepts vus en cours.

**Objectif général** : Dans ce TP (et les suivants), les exercices devront combiner travail théorique (calculs) et implémentation pratique.

---
### Rappels théoriques
Pour les questions de calcul, nous utiliserons les notations suivantes :
- **Espérance mathématique** : $E[f(X)]=<f(X)>$ pour une variable aléatoire $X$ et une fonction $f$. On notera dans certain cas $<f(X)>_{X\simP}$ pour rappeler que la loi de $X$ est $P$. 
- **Relation clé** : Pour deux variables aléatoires $X$ et $Y$, on a :
  $$<XY> = <X><Y> + \text{Cov}(X, Y)$$
  *Si $X$ et $Y$ sont **indépendantes**, alors $\text{Cov}(X, Y) = 0$.*
- **Moments centrés** On notera ainsi les moments centrés $E[(X-E[X])^n]=<X^n>_c$

**Outils Python** :
- **`numpy`** : Pour les tableaux numériques et les opérations mathématiques.
- **`numpy.random`** : Pour les tirages aléatoires.
- **`matplotlib`** : Pour les visualisations graphiques.

**Précision des et affichage des résultats** (ces recommandations sont toujours valables !):
- **Troncature des valeurs numériques** : Si possible, suivre les recommandations du *Particle Data Group* ([page 13 du Booklet](https://pdg.lbl.gov/2011/reviews/rpp2011-rev-rpp-intro.pdf)).
- **Présentation des graphiques** :
  - **Barres d’erreur** : Dans la mesure du possible les inclure pour visualiser l’incertitude.
  - **Choix des axes** : Adapter les échelles pour une lecture claire.
  - **Comparaisons** : Utiliser des différences ou rapports pour mettre en évidence les dynamiques fortes.

---
### Exercice : Intégrale Monte Carlo du volume d’une ellipse en *n* dimensions
La méthode Monte Carlo permet d’estimer le volume d’une figure en tirant des points aléatoires dans un volume englobant et en corrigeant par la fraction de points inclus dans la figure.

**Formulation mathématique** :
Soit $\Omega$ un volume et $\mathbb{1}^\Omega(x)$ son **indicatrice** (valant $1$ si $x \in \Omega$, $0$ sinon). Le volume $V^\Omega$ s’écrit :
$$
V^\Omega = V \times <\mathbb{1}^\Omega(x)>_{x \sim U} = V \times \int \mathbb{1}^\Omega(x) \, dU(x)
$$
où $U$ est une **distribution uniforme** sur un volume englobant de volume $V$. On rappelle que $<1>_{x~U}=\int  dU(x) = 1$.

**Approximation par Monte Carlo** :
Pour $N$ échantillons tirés selon $U$, on approche $V^\Omega$ par :
$$
V^\Omega_N = \frac{V}{N} \sum_{i=1}^N \mathbb{1}^\Omega(x_i)
$$
*Remarque* : $V^\Omega_N$ est un estimateur de  $V^\Omega$.

---
### Questions et implémentation
1. **Vérification théorique** :
   Montrer par le calcul que $V^\Omega_N$ est un estimateur sans biais de $V^\Omega$, c’est-à-dire que :
   $$<V^\Omega_N> = V^\Omega$$

2. **Implémentation en Python** :
   Écrire une fonction `volume_ellipse` qui prend en entrée :
   - Un tableau `numpy` de taille `m` (demi-axes de l’ellipse en *m* dimensions).
   - Un entier `N` (nombre de points aléatoires).
   *Retourne* : Le volume estimé de l’ellipse.
   *Conseil* : Créer une fonction auxiliaire `indicatrice` qui, pour un tableau de taille `[N, m]` (coordonnées de $N$ points en *m* dimensions), retourne une liste de $0$ ou $1$ (selon l’appartenance à l’ellipse).

3. **Test en 2D** :
   - Choisir un grand axe et un petit axe (ex: $a=3$, $b=2$).
   - Tester la fonction avec $N=1000$ et comparer au résultat théorique ($\pi ab$).
   - L’estimation est-elle exacte ? Pourquoi ?

4. **Étude statistique** :
   - Réaliser $L=500$ estimations de l’aire de la même ellipse.
   - Tracer un histogramme des résultats.
   - Forme attendue : Justifier la distribution observée pour $V^\Omega_{1000}$.


5. **Écart-type et probabilité** :
   - Calculer la probabilité que $V^\Omega_{1000}$ s’écarte de la valeur vraie de plus d’un écart-type ($\sigma$).
   - Estimer cette probabilité à partir des $500$ simulations et comparer au résultat théorique.

6. **Variance de l’estimateur** :
   - Évaluer la variance de $V^\Omega_{1000}$.
   - Montrer que l’estimateur naïf :
     $$
     \text{Var}_{\text{naïf}}(V^\Omega_{1000}) = \frac{1}{L} \sum_{j=1}^L (V^\Omega_{1000,j} - \bar{V^\Omega_{1000}})^2
     $$
     est biaisé (où $\bar{V^\Omega_{1000}}$ est la moyenne empirique).
   - Proposer un estimateur non biaisé et vérifier l’effet du biais en calculant la variance pour $L=10, 50, 100, 200$.
   - Visualisation : Tracer les valeurs et le biais attendu sur un même graphique.

7. **Comportement asymptotique** :
   - D'après la théorie, comment la variance de $V^\Omega_N$ décroît-elle avec $N$ ?
   - Vérification :
     - Estimer l’aire de l’ellipse pour différentes valeurs de $N$.
     - Tracer la variance en fonction de $N$ et comparer au comportement théorique.
     - *Précision* : Pour $N$ grand, l’erreur sur la variance empirique est $\sigma/\sqrt{2N}$ (où $\sigma$ est l’écart-type). Ajouter des barres d’erreur au graphique.

8. **Précision et nombre de tirages** :
   - Combien de tirages $N$ sont nécessaires pour une précision de 1% ? 0.1% ?
   - Calculer le volume d’une ellipse en 10 dimensions avec des demi-axes `[1, 3, 2, 5, 6, 1, 2, 0.5, 2, 4]`.



---
## Deuxième partie Transformée de Fourier


Dans cette partie, on aborde quelques aspects de la transfomrée de Fourier et de son utilité pour l'astrophysique. C'est une application de la première partie du second cours.


In [9]:
import numpy as np
import matplotlib.pyplot as plt

### Echantillonnage

1. Ecrire une fonction python `generateur_de_fonction` ayant comme paramètre $\nu$, une fréquence, et qui renvoie la fonction $\sin (2\pi \nu \times t)$ où $t$ est la variable. 
2. Pour $\nu = 0.5\; s^{-1}$ , définir à l'aide du générateur précédent une fonction `y` de paramètre $t$ qui renvoie $y(t) = \sin (2\pi \nu\times t)$. 
3. Echantillonner $y$ sur $t\in [0,20]$ s  avec une période d'échantillonnage $T_e = 0.2$ s et tracer `yech` en fonction de `tech`.


### FFT
1. Executer np.fft.fft sur le tableau `yech` pour obtenir la transformée de Fourier discrète `tfd`, que l'on prendra soin de normaliser par `len(yech)` (NB: sans mot-clé de normalisation "norm", np.fft.fft renvoie un tableau non normalisé. Si on utilise "norm = 'ortho'", la tf est divisée par \sqrt(n). Si on utilise "norm = 'forward'", la tf est divisée par n.)
2. Tracer les parties réelles et imaginaires `tfd.real` et `tfd.imag`, ainsi que la puissance `np.abs(tdf)**2` et commentez ce que vous voyez : pourquoi cette symétrie ? où est la fréquence minimale, maximale et combien valent ces fréquences, combien ce tableau a-t-il d'éléments ?
3. Construire à la main le tableau de fréquences correspondant, comparer au résulat de `np.fft.fftfreq(len(tfd), d = tech[1]-tech[0])` et tracer la densité spectrale de puissance en fonction de la fréquence.

La fonction `get_fft_pow` suivante prend pour arguments deux tableaux sembables à `tech` et `yech`, et renvoie deux tableaux : les fréquences triés par ordre croissant et la densité spectrale de puissance |TF(yech)|$^2$. 
Notez l'utilisation  des routines np.fft.fft, np.fft.fftfreq, (et `np.fft.fftshift` pour réordonner les tableaux).

In [12]:
def get_fft_pow(t, s, norm = 'forward'):
  
    timestep = t[1]-t[0]
    freq = np.fft.fftfreq(len(s), d = timestep)
    tf = np.fft.fft(s, norm = norm)     # norm = 'forward' =>  tf *= 1/len(t) ; norm = 'ortho' => tf *= 1/sqrt(len(t))
    freq = np.fft.fftshift(freq)
    tf = np.fft.fftshift(tf)
    tfpow = np.abs(tf)**2  

    return (freq, tfpow)

Utilisation de cette fonction:

In [ ]:
f_ord, tfd_ord = get_fft_pow(tech, yech)
plt.plot(f_ord,tfd_ord)

4. Avec la TF inverse (`np.fft.ifft`), reconstruire le signal initial.
5. Vérifier que la TF inverse de `tfd` $\times \exp(i2\pi \nu t_0)$ où $t_0 = 1 s$ donne bien un signal décalé temporellement de $t_0$. Cette opération a-t-elle agi sur l'amplitude ou la phase de `tdf` ?


6. Tracer le signal, et la TF de la fonction `ydouble` suivante échantillonnée sur [0,20] avec Te = 0.2:

In [15]:
ydouble = lambda x:np.sin(2*np.pi*0.5*x) + 0.5* np.cos(2*np.pi*0.3*x)

Reconstruction : Sous réserve que l'échantillonnage soit suffisamment bon, le signal original peut être reconstruit à une résolution arbitrairement élevée à partir du signal échantillonné :


In [ ]:
def enhance(tech, yech, tout):
    ''' Reconstruit le signal yech(tech) à la résolution de tout avec une somme de sinus cardinaux.
    Hypothèse sous-jacente : la fréquence d'échantillonage est supérieure au double de la fréquence 
    de coupure du signal (théorème de Nyquist-Shannon) '''
    
    yout = np.zeros_like(tout)
    Te = tech[1] - tech[0]
    for i in range(len(yech)):
        yout += yech[i] * np.sinc((tout-i*Te)/Te)
    return yout

# demo:
tpur = np.arange(0,20,0.01)
ypur = ydouble(tpur)

# signal échantillonné
Te = 0.8
tech = np.arange(0,20,Te)
yech = ydouble(tech)

# signal reconstruit à plus haute résolution
tout = np.arange(0,20,Te/10)
yout = enhance(tech, yech, tout)

plt.figure(figsize=(20,12))
plt.plot(tpur, ypur, label = 'signal "pur"')
plt.plot(tech, yech, 'o-', label = 'signal echantilloné')
plt.plot(tout, yout, label = 'signal reconstruit à une plus haute résolution que le signal échantilloné')
plt.legend()
plt.show()

### Théorème de Nyquist-Shannon
7. Refaire les opérations précédentes (échantillonnage, TF, examen de la TF) avec une sinusoide de fréquence $\nu$ = 0.5, pour $T_e$ valant successivement 0.5, 1, 1.5, et 3. Que constate-t-on sur les fréquences mesurées dans la TFD si $\nu_e= 1/T_e$ devient inférieur au double de la fréquence $\nu=0.5$ Hz du signal ?


# Vitesses radiales

On veut analyser la série temporelle contenue dans le fichier `radial.npz` qui représente la vitesse radiale mesurées sur une étoile. 

1. Lire le fichier pour obtenir les variables `t` et `v` . Le temps est en jours et la vitesse en m/s.
2. Tracer le signal
3. Calculer et observer sa TFD. Que constate-t-on ?
5. Quelles sont les fréquences cachées dans ce signal ?
4. Filter le signal en mettant à zéro les éléments de la TFD en dessous d'une certaine puissance
6. Reconstruire le signal filtré
7. En utilisant une somme de sinus cardianaux pondérés par les échantillons du signal reconstruit, tracer avec une bonne résolution temporelle (échantillonnage en temps de 0.02 s) la fonction correspondant au signal filtré, et comparer cette reconstruction au signal pur original (tableaux `t_pur` et `signal_pur` du fichier `radial_npz`).